In [ ]:
from imutils import face_utils
from imutils.face_utils import rect_to_bb
import imutils
import dlib
import numpy as np
import cv2
import os
from os import listdir
from os.path import isfile, join

In [ ]:
mypath = "../images/prev_class/AVR_data/"
directories = [dir for dir in listdir(mypath)]
print(directories)
files = []
for dir in directories :
    files_of_person = [f for f in listdir(mypath + dir)]
    files.append(files_of_person)
print(files)

In [ ]:
def scaleImage(predictor, image, gray, rect) :
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    (w, h) = image.shape[:2]
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    leftEyePts = shape[lStart:lEnd]
    rightEyePts = shape[rStart:rEnd]
    leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
    rightEyeCenter = rightEyePts.mean(axis=0).astype("int")
    dY = rightEyeCenter[1] - leftEyeCenter[1]
    dX = rightEyeCenter[0] - leftEyeCenter[0]
    angle = np.degrees(np.arctan2(dY, dX)) - 180
    dist = np.sqrt((dX ** 2) + (dY ** 2))
    desiredDist = 128 
    scale = desiredDist / dist
    eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) // 2,
        (leftEyeCenter[1] + rightEyeCenter[1]) // 2)
    M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)
    tX = w * 0.5
    tY = h * 0.35
    M[0, 2] += (tX - eyesCenter[0])
    M[1, 2] += (tY - eyesCenter[1])
    output = cv2.warpAffine(image, M, (w, h),flags=cv2.INTER_CUBIC)
    return output

In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../resources/shape_predictor_68_face_landmarks.dat')

In [ ]:
for i in range(len(files)) :
    images = files[i]
    for imageName in images :
        filePath = mypath + directories[i] + '/' + imageName
        image = cv2.imread(filePath)
        image = imutils.resize(image, width=500)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 2)
        for rect in rects:
            (x, y, w, h) = rect_to_bb(rect)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.imshow("Output", image)
            faceAligned = scaleImage(predictor,image,gray,rect)
            dirName = "../images/alignedImages/" + directories[i] +'/'
            if not os.path.exists(os.path.dirname(dirName)):
                os.makedirs(os.path.dirname(dirName))
            else : cv2.imwrite(dirName + imageName , faceAligned)